In [39]:
import numpy as np
from ldpc import bposd_decoder
from ldpc.codes import rep_code 
from bposd.hgp import hgp
import tqdm
import time
import nbimporter
import BP_OSD_Simulation_V2 as my_bposd

In [8]:
h = rep_code(6)
surface_code=hgp(h1=h,h2=h,compute_distance=True)
surface_code.test()

<Unnamed CSS code>, (2,4)-[[61,1,6]]
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params (2,4)-[[61,1,6]]


True

In [9]:
bpd=bposd_decoder(
    surface_code.hz,#the parity check matrix
    error_rate=0.08,
    channel_probs=[None], #assign error_rate to each qubit. This will override "error_rate" input variable
    max_iter=surface_code.N, #the maximum number of iterations for BP)
    bp_method="ms",
    ms_scaling_factor=0, #min sum scaling factor. If set to zero the variable scaling factor method is used
    osd_method="osd_cs", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
    osd_order=7 #the osd search depth
    )

In [38]:
error=np.random.randint(2, size=surface_code.N)
while(sum(error) == 0):
    error = np.random.randint(2, size=surface_code.N)
syndrome=surface_code.hz@error %2
bpd.decode(syndrome)

print("Error")
print(error)
print("BP+OSD Decoding")
print(bpd.osdw_decoding)
#Decoding is successful if the residual error commutes with the logical operators

residual_error=(bpd.osdw_decoding+error) %2
a=(surface_code.lz@residual_error%2).any()
if a: a="Yes"
else: a="No"
print(f"Logical Error: {a}\n")

print(a)

Error
[1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 1 1 0 0 0 0 1 1 1 1 0
 1 1 0 1 0 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1]
BP+OSD Decoding
[0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]
Logical Error: No

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
num_iter = 2000
codespace_projection = [0]*num_rounds
logical_errors = [0]*num_rounds
uncorrected_logical_error_rate = [0]*num_rounds
error_probability = 0.08
count = 0

for r in tqdm(range(num_iter)):
    # Simulate error
    error=np.random.randint(2, size=surface_code.N)
    while(sum(error) == 0):
        error = np.random.randint(2, size=surface_code.N)

    # Get syndromes
    syndrome_x = surface_code.hz@error % 2
    syndrome_z = surface_code.hx@error % 2
    bpd.decode(syndrome_x)
    bpd.decode(syndrome_z)
    